In [1]:
!pip install yfinance
!pip install seaborn
!pip install mplcursors
!pip install Flask
!pip install flask_ngrok
!pip install line-bot-sdk
!pip install ta

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29422 sha256=1ebc7b249b42faab47091b47f05076487aa9f44b788f6ae629d72660b530c3b5
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\a1\d7\29\7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
Successfully built ta


In [2]:
#整合
import os
import json
import requests
from flask import Flask, request, abort, session, jsonify
from flask_ngrok import run_with_ngrok
from linebot import LineBotApi, WebhookHandler
import yfinance as yf
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from linebot.exceptions import InvalidSignatureError
from bs4 import BeautifulSoup
from linebot.models import FollowEvent, JoinEvent, MessageEvent, TextMessage, TextSendMessage

#股票資訊
access_token = 'mq7+Frak/yl4cYOol+orRrD6QIZOxVvl9U5uhvBO0K+9wyes/G1cLMgl0EPv0exK0t/603JyrtQ2qPttQgkzXAqg0+fxkBrCGYOgxvLKZPpEcH05ZnG6oXGrPx9kpzn0+KcQ+jfO4qfpT5evVlO6pgdB04t89/1O/w1cDnyilFU='
secret = 'b0059f8cb5ca4077ff1ee4ae0efecc1e'

# 初始化 Flask
app = Flask(__name__)
run_with_ngrok(app)

# 初始化 LINE Bot
line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(secret)

# 設置初始股票代碼和月份
stock_symbol = ''
months = 0

# 處理 LINE Bot 的 Webhook
@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)

    try:
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)

        for event in json_data['events']:
            if event['type'] == 'message' and event['message']['type'] == 'text':
                global stock_symbol, months
                user_message = event['message']['text']

                if user_message == '股票資訊':
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text="請使用以下格式喲!：\nex:\n股票代碼:2330.TW\n查詢月份數:6"))
                elif '股票代碼:' in user_message and '查詢月份數:' in user_message:
                    stock_symbol_start = user_message.find('股票代碼:') + len('股票代碼:')
                    stock_symbol_end = user_message.find('查詢月份數:')
                    stock_symbol = user_message[stock_symbol_start:stock_symbol_end].strip()

                    months_start = user_message.find('查詢月份數:') + len('查詢月份數:')
                    months = int(user_message[months_start:].strip())

                    stock_info = scrape_stock_data(stock_symbol, months)
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text=stock_info))
                elif user_message.startswith('股票代碼:'):
                    stock_symbol = user_message.replace('股票代碼:', '')
                elif user_message.startswith('查詢月份數:'):
                    months = int(user_message.replace('查詢月份數:', ''))
                else:
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text="如有什麼提問可以從我們的選單中查詢呦~"))

    except Exception as e:
        print('Error:', str(e))

    return 'OK'

# 計算 KD 指標的函數
def calculate_kd(stock_data, window=14):
    high_prices = stock_data['High'].rolling(window=window, min_periods=1).max()
    low_prices = stock_data['Low'].rolling(window=window, min_periods=1).min()

    rsv = (stock_data['Close'] - low_prices) / (high_prices - low_prices) * 100
    stock_data['K'] = rsv.ewm(span=3, adjust=False).mean()
    stock_data['D'] = stock_data['K'].ewm(span=3, adjust=False).mean()

# 爬取股票數據的函数
def scrape_stock_data(stock_symbol, months):
    try:
        # 計算起始日期和结束日期
        start_date = datetime.datetime.today().replace(day=1) - datetime.timedelta(days=30 * months)
        end_date = datetime.datetime.today()  # 结束日期為今天

        # 使用 yfinance 抓取股票數據
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

        # 計算 KD 指標
        calculate_kd(stock_data)

        # 爬取數據生成股票URL
        base_url = "https://tw.stock.yahoo.com/"
        exchange = "quote"

        # 計算 MACD 和 Signal 線
        shortEMA = stock_data['Close'].ewm(span=12, adjust=False).mean()
        longEMA = stock_data['Close'].ewm(span=26, adjust=False).mean()
        MACD = shortEMA - longEMA
        signal = MACD.ewm(span=9, adjust=False).mean()

        # 計算線性回歸趨勢線
        x = np.arange(len(stock_data)).reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(x, stock_data['Close'])
        trend_line = regressor.predict(x)

        # 設置交易門槛
        buy_threshold = 0.0

        # 判断是否適合買進
        current_macd = MACD[-1]
        current_price = stock_data['Close'][-1]
        current_k = stock_data['K'][-1]
        current_d = stock_data['D'][-1]

        if current_macd > buy_threshold and current_price > trend_line[-1] and current_k > current_d:
            buy_signal = "可以買! \n原因：MACD 高於門檻、股價高於趨勢線，且KD指標向上"
        else:
            buy_signal = "建議先不買呦。\n原因：MACD 低於門檻或股價低於趨勢線，或KD指標向下"

        # 計算收盤價的變化
        price_diff = stock_data['Close'].diff()
        # 確定線的顏色
        line_color = ['red' if diff > 0 else 'green' for diff in price_diff]

        # 繪製股價、趨勢線、MACD 和 Signal
        plt.figure(figsize=(12, 6))

        # 股價和趨勢線
        plt.subplot(2, 1, 1)
        plt.plot(stock_data.index, stock_data['Close'], label='Close Price', color='black')
        plt.plot(stock_data.index, trend_line, label='Trend Line', color='gray', linestyle='dashed')

        for i in range(1, len(stock_data)):
            plt.plot([stock_data.index[i - 1], stock_data.index[i]], [stock_data['Close'][i - 1], stock_data['Close'][i]], color=line_color[i - 1])

        plt.title('Stock Price and Trend Line')
        plt.xlabel('Date')
        plt.ylabel('Price/Value')
        plt.legend()
        plt.xticks(rotation=45)

        # KD圖
        plt.subplot(2, 1, 2)
        plt.plot(stock_data.index, stock_data['K'], label='K', color='blue')
        plt.plot(stock_data.index, stock_data['D'], label='D', color='orange')

        plt.title('KD Line')
        plt.xlabel('Date')
        plt.ylabel('KD Value')
        plt.legend()
        plt.xticks(rotation=45)

        plt.tight_layout()

        # 保存股票图像
        plt.savefig('stock.png')
        plt.close()

        # 上传股票图像到Imgur并获取图像URL
        image_url = upload_image_to_imgur('stock.png', client_id)
        if image_url:
            stock_info = f"股票代碼：{stock_symbol}\n"
            stock_info += f"查詢月份數：{months} 個月\n\n"
            stock_info += f"當前股價：{current_price:.2f}\n"
            stock_info += f"當前MACD：{current_macd:.2f}\n"
            stock_info += f"當前KD：K={current_k:.2f}, D={current_d:.2f}\n"
            stock_info += f"趨勢線：{trend_line[-1]:.2f}\n\n"
            stock_info += f"建議是否買入：{buy_signal}\n\n"
            stock_info += f"圖片URL：{image_url}\n\n"
            stock_info += f"{base_url}{exchange}/{stock_symbol}/"
        else:
            stock_info = "無法獲取股票圖像"

        return stock_info

    except Exception as e:
        return str(e)


def upload_image_to_imgur(image_path, client_id):
    url = "https://api.imgur.com/3/image"
    headers = {
        "Authorization": f"Client-ID {client_id}"
    }
    files = {
        'image': open(image_path, 'rb')
    }
    response = requests.post(url, headers=headers, files=files)
    if response.status_code == 200:
        data = json.loads(response.text)
        image_url = data['data']['link']
        return image_url
    else:
        print(f"Failed to upload image. Status code: {response.status_code}")
        return None

client_id = '619264d0287bbaf'

#通知設定
line_bot_api = LineBotApi('mq7+Frak/yl4cYOol+orRrD6QIZOxVvl9U5uhvBO0K+9wyes/G1cLMgl0EPv0exK0t/603JyrtQ2qPttQgkzXAqg0+fxkBrCGYOgxvLKZPpEcH05ZnG6oXGrPx9kpzn0+KcQ+jfO4qfpT5evVlO6pgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('b0059f8cb5ca4077ff1ee4ae0efecc1e')
#user_id = ('U48fd8377e3cf3121e3f355f8d301fe10')  # 替換為你的 User ID

# 定義 send_notification 函數
def send_notification(user_id,message):
    try:
        line_bot_api.push_message(user_id, TextSendMessage(text=message))
    except Exception as e:
        print(f"Error sending notification: {e}")


# 設定股票觸發價格
trigger_price = None
current_stock_symbol = None
current_price = None

def get_stock_price(symbol):
    try:
        # 使用 yfinance 獲取股票價格
        stock_data = yf.Ticker(symbol)
        latest_close = stock_data.history(period='1d')['Close'].iloc[-1]
        return latest_close
    except Exception as e:
        print(f"Error getting stock price: {e}")
        return None

# 在全域變數中儲存每隻股票的高標和低標，以使用者 ID 作為鍵
user_stock_info = {}
first_interaction = {}

# Line Bot 訊息處理
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 在 Line Bot 訊息處理的地方進行修改
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global user_stock_info
    global first_interaction
    user_id = event.source.user_id
    user_stocks = user_stock_info.get(user_id, {})
    user_input = event.message.text

    if user_input.lower() == '通知設定':
        line_bot_api.push_message(user_id, TextSendMessage(text="您好！歡迎使用股票觸發價格提醒機器人\U0001F60A\n可輸入查看設定\u2192設定查看目前設定\n可輸入重設\u2192重設目前設定\n可輸入刪除\u2192刪除想刪除的股票代碼(刪除,股票代碼)\n請輸入股票代碼(股票代碼,高標,低標)："))
        return
    if user_input.lower() == '刪除':
        line_bot_api.push_message(user_id, TextSendMessage(text="請輸入要刪除的股票代碼 ex:刪除2330.TW"))
        return

    user_stocks = user_stock_info.get(user_id, {})

    if user_input.startswith('刪除'):
        manage_deletion(event, user_stock_info)
        return

    if user_input.lower() == '查看設定':
        # 顯示目前使用者的股票設定
        response_text = "目前的股票設定：\n"
        for stock, (high_threshold, low_threshold) in user_stock_info.get(user_id, {}).items():
            response_text += f"{stock} - 高標：{high_threshold}，低標：{low_threshold}\n"
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=response_text)
        )
        return

    if user_input.lower() == '重設':
        # 重設使用者的股票設定
        user_stock_info[user_id] = {}
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="已重設股票觸發價格設定。請重新輸入股票代碼、高標和低標：")
        )
        return

    # 解析使用者輸入的股票代碼、高標和低標
    input_parts = [part.strip() for part in user_input.split(',')]
    if len(input_parts) == 3:
        stock_symbol, high_threshold, low_threshold = map(str.strip, input_parts)
        user_stock_info.setdefault(user_id, {})[stock_symbol] = (float(high_threshold), float(low_threshold))

        current_price = get_stock_price(stock_symbol)

        if current_price is not None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=f"已設定 {stock_symbol} 股票的高標為 {high_threshold}，低標為 {low_threshold}。當前價格為 {current_price}。")
            )
            # 如果當前價格達到或超過高標或低標，發送通知
            if current_price >= float(high_threshold):
                send_notification(user_id,f"{stock_symbol} 已達到高標 {high_threshold}，當前價格為 {current_price}")
            elif current_price <= float(low_threshold):
                send_notification(user_id,f"{stock_symbol} 已達到低標 {low_threshold}，當前價格為 {current_price}")
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="輸入格式不正確。請使用「股票代碼,高標,低標」的格式。")
            )
def manage_deletion(event, user_stock_info):
    user_id = event.source.user_id
    user_stocks = user_stock_info.get(user_id, {})
    user_input = event.message.text

    # 分割用户输入的消息
    parts = user_input.split(',')
    
    # 確保訊息正確
    if len(parts) != 2 or parts[0].strip() != '刪除':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="輸入格式不正確。請使用「刪除,股票代碼」的格式。")
        )
        return

    # 獲取股票代號
    stock_symbol = parts[1].strip()

    if stock_symbol in user_stocks:
        del user_stocks[stock_symbol]
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=f"已成功刪除 {stock_symbol} 的觸發點。")
        )
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=f"您沒有設定 {stock_symbol} 觸發點，無須刪除。")
        )
        

    print(f"Current state: current_stock_symbol={current_stock_symbol}, current_price={current_price}, trigger_price={trigger_price}")

# 主程式
if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\user\AppData\Local\Temp\ipykernel_26892\4279178768.py:28: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
C:\Users\user\AppData\Local\Temp\ipykernel_26892\4279178768.py:29: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)
C:\Users\user\AppData\Local\Temp\ipykernel_26892\4279178768.py:206: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = 

 * Running on http://db3a-123-240-92-189.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


In [ ]:
#整合
import os
import json
import requests
from flask import Flask, request, abort, session, jsonify
from flask_ngrok import run_with_ngrok
from linebot import LineBotApi, WebhookHandler
import yfinance as yf
import datetime
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.linear_model import LinearRegression
from linebot.exceptions import InvalidSignatureError
from bs4 import BeautifulSoup
from linebot.models import FollowEvent, JoinEvent, MessageEvent, TextMessage, TextSendMessage
import ta

#股票資訊
access_token = 'mq7+Frak/yl4cYOol+orRrD6QIZOxVvl9U5uhvBO0K+9wyes/G1cLMgl0EPv0exK0t/603JyrtQ2qPttQgkzXAqg0+fxkBrCGYOgxvLKZPpEcH05ZnG6oXGrPx9kpzn0+KcQ+jfO4qfpT5evVlO6pgdB04t89/1O/w1cDnyilFU='
secret = 'b0059f8cb5ca4077ff1ee4ae0efecc1e'

# 初始化 Flask
app = Flask(__name__)
run_with_ngrok(app)

# 初始化 LINE Bot
line_bot_api = LineBotApi(access_token)
handler = WebhookHandler(secret)

# 設置初始股票代碼和月份
stock_symbol = ''
months = 0

# 處理 LINE Bot 的 Webhook
@app.route("/", methods=['POST'])
def linebot():
    body = request.get_data(as_text=True)
    json_data = json.loads(body)

    try:
        signature = request.headers['X-Line-Signature']
        handler.handle(body, signature)

        for event in json_data['events']:
            if event['type'] == 'message' and event['message']['type'] == 'text':
                global stock_symbol, months
                user_message = event['message']['text']

                if user_message == '股票資訊':
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text="請輸入以下格式\U0001F917\nex:\n股票代碼:2330.TW\n查詢月份數:6"))
                elif '股票代碼:' in user_message and '查詢月份數:' in user_message:
                    stock_symbol_start = user_message.find('股票代碼:') + len('股票代碼:')
                    stock_symbol_end = user_message.find('查詢月份數:')
                    stock_symbol = user_message[stock_symbol_start:stock_symbol_end].strip()

                    months_start = user_message.find('查詢月份數:') + len('查詢月份數:')
                    months = int(user_message[months_start:].strip())

                    stock_info = scrape_stock_data(stock_symbol, months)
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text=stock_info))
                elif user_message.startswith('股票代碼:'):
                    stock_symbol = user_message.replace('股票代碼:', '')
                elif user_message.startswith('查詢月份數:'):
                    months = int(user_message.replace('查詢月份數:', ''))
                else:
                    line_bot_api.reply_message(event['replyToken'], TextSendMessage(text="如有什麼提問可以從我們的選單中查詢呦~"))

    except Exception as e:
        print('Error:', str(e))

    return 'OK'


# 計算 KD 指標的函數
def calculate_kd(stock_data, window=14):
    high_prices = stock_data['High'].rolling(window=window, min_periods=1).max()
    low_prices = stock_data['Low'].rolling(window=window, min_periods=1).min()

    rsv = (stock_data['Close'] - low_prices) / (high_prices - low_prices) * 100
    stock_data['K'] = rsv.ewm(span=3, adjust=False).mean()
    stock_data['D'] = stock_data['K'].ewm(span=3, adjust=False).mean()

# 計算 RSI 指標的函數
def calculate_rsi(stock_data, window=14):
    stock_data['RSI'] = ta.momentum.rsi(stock_data['Close'], window=window)

# 爬取股票數據的函数
def scrape_stock_data(stock_symbol, months):
    try:
        # 計算起始日期和结束日期
        start_date = datetime.datetime.today().replace(day=1) - datetime.timedelta(days=30 * months)
        end_date = datetime.datetime.today()  # 结束日期為今天

        # 使用 yfinance 抓取股票數據
        stock_data = yf.download(stock_symbol, start=start_date, end=end_date)

        # 計算 KD 和 RSI 指標
        calculate_kd(stock_data)
        calculate_rsi(stock_data)

        # 爬取數據生成股票URL
        base_url = "https://tw.stock.yahoo.com/"
        exchange = "quote"

        # 計算 MACD 和 Signal 線
        shortEMA = stock_data['Close'].ewm(span=12, adjust=False).mean()
        longEMA = stock_data['Close'].ewm(span=26, adjust=False).mean()
        MACD = shortEMA - longEMA
        signal = MACD.ewm(span=9, adjust=False).mean()

        # 計算線性回歸趨勢線
        x = np.arange(len(stock_data)).reshape(-1, 1)
        regressor = LinearRegression()
        regressor.fit(x, stock_data['Close'])
        trend_line = regressor.predict(x)

        # 設置交易門槛
        buy_threshold = 0.0

        # 判断是否適合買進
        current_macd = MACD[-1]
        current_price = stock_data['Close'][-1]
        current_k = stock_data['K'][-1]
        current_d = stock_data['D'][-1]

        if current_macd > buy_threshold and current_price > trend_line[-1] and current_k > current_d:
            buy_signal = "可以買!\U0001F60D\n原因：MACD 高於門檻、股價高於趨勢線，且KD指標向上"
        else:
            buy_signal = "建議先不買呦\U0001F62D\n原因：MACD 低於門檻或股價低於趨勢線，或KD指標向下"

        # 計算收盤價的變化
        price_diff = stock_data['Close'].diff()
        # 確定線的顏色
        line_color = ['red' if diff > 0 else 'green' for diff in price_diff]

        # 繪製股價、趨勢線、MACD 和 Signal
        plt.figure(figsize=(12, 20))


        # 股價和趨勢線
        plt.subplot(3, 1, 1)
        plt.plot(stock_data.index, stock_data['Close'], label='Close Price', color='black')
        plt.plot(stock_data.index, trend_line, label='Trend Line', color='gray', linestyle='dashed')

        for i in range(1, len(stock_data)):
            plt.plot([stock_data.index[i - 1], stock_data.index[i]], [stock_data['Close'][i - 1], stock_data['Close'][i]], color=line_color[i - 1])

        plt.title('Stock Price and Trend Line')
        plt.xlabel('Date')
        plt.ylabel('Price/Value')
        plt.legend()
        plt.xticks(rotation=45)

        # KD 圖
        plt.subplot(3, 1, 2)
        plt.plot(stock_data.index, stock_data['K'], label='K', color='blue')
        plt.plot(stock_data.index, stock_data['D'], label='D', color='orange')
        plt.title('KD Line')
        plt.xlabel('Date')
        plt.ylabel('KD Value')
        plt.legend()
        plt.xticks(rotation=45)

        # RSI 圖
        plt.subplot(3, 1, 3)
        plt.plot(stock_data.index, stock_data['RSI'], label='RSI', color='purple')
        plt.title('RSI Line')
        plt.xlabel('Date')
        plt.ylabel('RSI Value')
        plt.legend()
        plt.xticks(rotation=45)

        plt.tight_layout()

        # 保存股票图像
        plt.savefig('stock.png')
        plt.close()

        image_url = upload_image_to_imgur('stock.png', client_id)
        if image_url:
            stock_info = f"是否買入：{buy_signal}\n\n"
            stock_info += f"圖片URL：{image_url}\n\n"
            stock_info += f"當前股價：{current_price:.2f}\n"
            stock_info += f"當前MACD：{current_macd:.2f}\n"
            stock_info += f"當前KD：K={current_k:.2f}, D={current_d:.2f}\n"
            stock_info += f"{base_url}{exchange}/{stock_symbol}/"
        else:
            stock_info = "無法獲取股票圖像"

        return stock_info

    except Exception as e:
        return str(e)


def upload_image_to_imgur(image_path, client_id):
    url = "https://api.imgur.com/3/image"
    headers = {
        "Authorization": f"Client-ID {client_id}"
    }
    files = {
        'image': open(image_path, 'rb')
    }
    response = requests.post(url, headers=headers, files=files)
    if response.status_code == 200:
        data = json.loads(response.text)
        image_url = data['data']['link']
        return image_url
    else:
        print(f"Failed to upload image. Status code: {response.status_code}")
        return None

client_id = '619264d0287bbaf'

#通知設定
line_bot_api = LineBotApi('mq7+Frak/yl4cYOol+orRrD6QIZOxVvl9U5uhvBO0K+9wyes/G1cLMgl0EPv0exK0t/603JyrtQ2qPttQgkzXAqg0+fxkBrCGYOgxvLKZPpEcH05ZnG6oXGrPx9kpzn0+KcQ+jfO4qfpT5evVlO6pgdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('b0059f8cb5ca4077ff1ee4ae0efecc1e')

# 定義 send_notification 函數
def send_notification(user_id,message):
    try:
        line_bot_api.push_message(user_id, TextSendMessage(text=message))
    except Exception as e:
        print(f"Error sending notification: {e}")


# 設定股票觸發價格
trigger_price = None
current_stock_symbol = None
current_price = None

def get_stock_price(symbol):
    try:
        # 使用 yfinance 獲取股票價格
        stock_data = yf.Ticker(symbol)
        latest_close = stock_data.history(period='1d')['Close'].iloc[-1]
        return latest_close
    except Exception as e:
        print(f"Error getting stock price: {e}")
        return None

# 在全域變數中儲存每隻股票的高標和低標，以使用者 ID 作為鍵
user_stock_info = {}
first_interaction = {}

#推薦股
def get_taiwan_stock_codes():
    url = "https://www.twse.com.tw/exchangeReport/MI_INDEX20?response=json&date=&_=123456"
    response = requests.get(url)
    data = response.json()
    stocks = [(item[1], item[2]) for item in data['data']]
    return stocks

def get_us_stock_codes():
    url = "https://finance.yahoo.com/most-active"
    response = requests.get(url)
    if response.status_code == 200:
        stocks = []
        soup = BeautifulSoup(response.text, 'html.parser')
        for row in soup.find_all('tr', {'class': 'simpTblRow'}):
            symbol = row.find('td', {'aria-label': 'Symbol'}).text
            english_name = row.find('td', {'aria-label': 'Name'}).text
            chinese_name = english_name
            stocks.append((symbol, chinese_name))
        return stocks
    else:
        return []

def display_stocks(stocks):
    message = "熱門股前20名排行\U0001F60E\n\n"
    for code, name in stocks:
        message += f"{code}: {name}\n"
    return message

# Line Bot 訊息處理
@app.route("/", methods=['POST'])
def callback():
    signature = request.headers['X-Line-Signature']
    body = request.get_data(as_text=True)

    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

# 在 Line Bot 訊息處理的地方進行修改
@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    global user_stock_info
    global first_interaction
    user_id = event.source.user_id
    user_stocks = user_stock_info.get(user_id, {})
    user_input = event.message.text

    if user_input.lower() == '通知設定':
        line_bot_api.push_message(user_id, TextSendMessage(text="您好！歡迎使用股票觸發價格提醒機器人\U0001F60A\n\n請輸入股票代碼(股票代碼,高標,低標)：\n查看設定\u2192設定查看目前設定\n重設\u2192重設目前設定\n刪除\u2192刪除想刪除的股票代碼(刪除,股票代碼)"))
        return
    if user_input.lower() == '刪除':
        line_bot_api.push_message(user_id, TextSendMessage(text="請輸入要刪除的股票代碼 ex:刪除,2330.TW"))
        return

    user_stocks = user_stock_info.get(user_id, {})

    if user_input.startswith('刪除'):
        manage_deletion(event, user_stock_info)
        return

    if user_input.lower() == '查看設定':
        # 顯示目前使用者的股票設定
        response_text = "目前的股票設定：\n"
        for stock, (high_threshold, low_threshold) in user_stock_info.get(user_id, {}).items():
            response_text += f"{stock} - 高標：{high_threshold}，低標：{low_threshold}\n"
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=response_text)
        )
        return

    if user_input.lower() == '重設':
        # 重設使用者的股票設定
        user_stock_info[user_id] = {}
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="已重設股票觸發價格設定。請重新輸入股票代碼、高標和低標：")
        )
        return

    # 如果以上條件都不符合，則處理其他訊息
    if user_input == "推薦股":
        stocks = get_taiwan_stock_codes()
        message = display_stocks(stocks)
        line_bot_api.reply_message(event.reply_token, TextSendMessage(text=message))
    
    # 解析使用者輸入的股票代碼、高標和低標
    input_parts = [part.strip() for part in user_input.split(',')]
    if len(input_parts) == 3:
        stock_symbol, high_threshold, low_threshold = map(str.strip, input_parts)
        user_stock_info.setdefault(user_id, {})[stock_symbol] = (float(high_threshold), float(low_threshold))

        current_price = get_stock_price(stock_symbol)

        if current_price is not None:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text=f"已設定 {stock_symbol} 股票的高標為 {high_threshold}，低標為 {low_threshold}。當前價格為 {current_price}。")
            )
            # 如果當前價格達到或超過高標或低標，發送通知
            if current_price >= float(high_threshold):
                send_notification(user_id,f"{stock_symbol} 已達到高標 {high_threshold}，當前價格為 {current_price}")
            elif current_price <= float(low_threshold):
                send_notification(user_id,f"{stock_symbol} 已達到低標 {low_threshold}，當前價格為 {current_price}")
        else:
            line_bot_api.reply_message(
                event.reply_token,
                TextSendMessage(text="輸入格式不正確。請使用「股票代碼,高標,低標」的格式。")
            )
def manage_deletion(event, user_stock_info):
    user_id = event.source.user_id
    user_stocks = user_stock_info.get(user_id, {})
    user_input = event.message.text

    # 分割用户输入的消息
    parts = user_input.split(',')
    
    # 確保訊息正確
    if len(parts) != 2 or parts[0].strip() != '刪除':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text="輸入格式不正確。請使用「刪除,股票代碼」的格式。")
        )
        return

    # 獲取股票代號
    stock_symbol = parts[1].strip()

    if stock_symbol in user_stocks:
        del user_stocks[stock_symbol]
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=f"已成功刪除 {stock_symbol} 的觸發點。")
        )
    else:
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text=f"您沒有設定 {stock_symbol} 觸發點，無須刪除。")
        )
        

    print(f"Current state: current_stock_symbol={current_stock_symbol}, current_price={current_price}, trigger_price={trigger_price}")

# 主程式
if __name__ == '__main__':
  app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:29: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi(access_token)
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:30: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler(secret)
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:218: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = 

 * Running on http://db3a-123-240-92-189.ngrok-free.app
 * Traffic stats available on http://127.0.0.1:4040


[*********************100%%**********************]  1 of 1 completed
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:123: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_macd = MACD[-1]
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:124: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  current_price = stock_data['Close'][-1]
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:125: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a va

Error: LineBotApiError: status_code=400, request_id=247c1c40-2fb1-4b2f-ae0a-d723612c0e12, error_response={"details": [], "message": "Invalid reply token"}, headers={'Server': 'legy', 'Content-Type': 'application/json', 'x-content-type-options': 'nosniff', 'x-frame-options': 'DENY', 'x-line-request-id': '247c1c40-2fb1-4b2f-ae0a-d723612c0e12', 'x-xss-protection': '1; mode=block', 'Content-Length': '33', 'Expires': 'Sun, 14 Apr 2024 14:50:59 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 14 Apr 2024 14:50:59 GMT', 'Connection': 'close'}


...

Error: LineBotApiError: status_code=400, request_id=e657d311-6286-4b83-8381-9ac5c9c50488, error_response={"details": [], "message": "Invalid reply token"}, headers={'Server': 'legy', 'Content-Type': 'application/json', 'x-content-type-options': 'nosniff', 'x-frame-options': 'DENY', 'x-line-request-id': 'e657d311-6286-4b83-8381-9ac5c9c50488', 'x-xss-protection': '1; mode=block', 'Content-Length': '33', 'Expires': 'Sun, 14 Apr 2024 14:51:20 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 14 Apr 2024 14:51:20 GMT', 'Connection': 'close'}


127.0.0.1 - - [14/Apr/2024 22:51:24] "POST / HTTP/1.1" 200 -
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:317: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(
127.0.0.1 - - [14/Apr/2024 22:51:26] "POST / HTTP/1.1" 200 -


Error: LineBotApiError: status_code=400, request_id=d9c91d6d-61b9-4132-afc1-fe20bbbf3b5f, error_response={"details": [], "message": "Invalid reply token"}, headers={'Server': 'legy', 'Content-Type': 'application/json', 'x-content-type-options': 'nosniff', 'x-frame-options': 'DENY', 'x-line-request-id': 'd9c91d6d-61b9-4132-afc1-fe20bbbf3b5f', 'x-xss-protection': '1; mode=block', 'Content-Length': '33', 'Expires': 'Sun, 14 Apr 2024 14:51:26 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 14 Apr 2024 14:51:26 GMT', 'Connection': 'close'}


127.0.0.1 - - [14/Apr/2024 22:51:29] "POST / HTTP/1.1" 200 -
C:\Users\user\AppData\Local\Temp\ipykernel_26892\3686416592.py:336: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(event.reply_token, TextSendMessage(text=message))
127.0.0.1 - - [14/Apr/2024 22:51:32] "POST / HTTP/1.1" 200 -


Error: LineBotApiError: status_code=400, request_id=955a277f-a6c3-4290-aa6c-08a4d956884a, error_response={"details": [], "message": "Invalid reply token"}, headers={'Server': 'legy', 'Content-Type': 'application/json', 'x-content-type-options': 'nosniff', 'x-frame-options': 'DENY', 'x-line-request-id': '955a277f-a6c3-4290-aa6c-08a4d956884a', 'x-xss-protection': '1; mode=block', 'Content-Length': '33', 'Expires': 'Sun, 14 Apr 2024 14:51:32 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 14 Apr 2024 14:51:32 GMT', 'Connection': 'close'}
